In [ ]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import year,month,hour, when, col, date_format, to_timestamp, round, coalesce
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
#sc = pyspark.SparkSession('local', 'test')
sc = pyspark.SparkContext(appName="HW3-Q1")
sqlContext = SQLContext(sc)

C:\Users\baris\anaconda3\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

  warnings.warn(


In [3]:
df = sqlContext.read.option("header",True).option("inferSchema",True) \
     .csv("pollution_us_2000_2016.csv")

In [4]:
df=df.withColumn("year",year("Date Local")).withColumn("month",month("Date Local"))  

In [5]:
dfmonth=df.select("State","County","year","month","NO2 Mean")\
.groupBy("State","County","year","month").avg("NO2 Mean")


In [6]:
df1=df.select("State","County","year","NO2 Mean")\
.groupBy("State","County","year").avg("NO2 Mean")

In [9]:
df1p=df1.toPandas()
#df1p.to_csv('df1.csv')

In [10]:
df2p=pd.read_csv("C:\\Users\\baris\\OneDrive\\desktop\\CSE 6242 Project\death_data.csv")

In [14]:
dfcal_ill=df1p[(df1p['State']=="California")|(df1p['State']=="Illinois")].sort_values(by=['year'])

In [15]:
df2p['State']=df2p['State'].map({'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA': 'California','CO':'Colorado',\
                                 'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','IA':'Iowa',\
                                 'ID':'Idaho','IL': 'Illinois','IN':'Indiana','KS':'Kansas','KY':'Kentucky','LA':'Louisiana' \
                                'MA':'Massachusetts','MD':'Maryland','ME':'Maine','MI':'Michigan','MN':'Minnesota','MO':'Missouri' \
                                'NC':'North Carolina','ND':'North Dakota','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico' \
                                'NV':'Nevada','NY':'New York','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania' \
                                'RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas' \
                                'UT':'Utah','VA':'Virginia','WA':'Washington','WI':'Wisconsin','WY':'Wyoming'})

In [16]:
df3p=pd.merge(dfcal_ill,df2p,how="inner", left_on=['State','County','year'], right_on=['State','County_Name','Year'])

In [17]:
df3p=df3p.T.drop_duplicates().T

In [69]:
def nit(df,year):
    
    dfyear=df3p[(df3p['year']==year)]  
    df_five_years=df3p[(df3p['year']>year-6)&(df3p['year']<year)]
    
    dfno2=df_five_years.groupby(["State","County"]).agg(NO2_fiveyears=pd.NamedAgg(column="avg(NO2 Mean)", aggfunc="mean")).reset_index()
    
    df_final=pd.merge(dfyear, dfno2, how="inner", on=['State','County'])
    
    return df_final

In [90]:
#nit(df3p,2015)

In [85]:
l=[]
for i in range(2006,2017):
    l.append(nit(df3p,i))

In [88]:
df4p=pd.concat(l,join="inner")

In [89]:
df4p.to_csv('df2.csv')